# Cleaning the Swiss Food Composition Dataset

This is an auxiliary notebook that documents the steps taken to clean the dataset from the [Swiss Food Composition Database](https://naehrwertdaten.ch/en/) and make it ready for use in the tutorials. It uses functions in the `clean_swiss_food_dataset.py` module found in the same folder.

For more information on the abbreviations of the dataset [see here](https://valeursnutritives.ch/en/abbreviations/).

## Setup

In [1]:
# importing required packages
import os
import requests

import clean_swiss_food_dataset

# setting jupyter notebook settings to reload custom packages
%load_ext autoreload
%autoreload 2

In [2]:
# URL of the dataset
url_data = "https://naehrwertdaten.ch/wp-content/uploads/2022/06/Swiss_food_composition_database.xlsx"

# path to store the raw data
path2data = "../data/swiss_food_composition_raw.xlsx"

## Fetch data

In [3]:
if not os.path.isfile(path2data):
    response = requests.get(url_data)

    with open(path2data, 'wb') as file:
        file.write(response.content)
else:
    print(f'File {path2data} already exists.')

File ../data/swiss_food_composition_raw.xlsx already exists.


## Clean data

In [4]:
dataset = clean_swiss_food_dataset.process_dataset(path2data)
dataset.head()

There are 196 cells containing non-numerical values. Stripping '<'. Done.
In total found 590 traces. Replacing them with 0.0001


/Users/anjaadamov/Documents/teaching/ml4food/ML4Food/scripts/clean_swiss_food_dataset.py:127: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 3:] = df.iloc[:, 3:].applymap(extract_value)


Done. Clean dataset ready.


,name,category,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,selenium_µg
0,Agar Agar,other,160.0,0.2,NaN,NaN,NaN,NaN,0.0,NaN,...,52.0,130.0,NaN,660.0,100.0,34.0,4.5,NaN,1.5,NaN
1,Agave syrup,sweets,293.0,0.0,0.0,NaN,NaN,NaN,73.1,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Almond,fruits,624.0,52.1,4.1,31.4,11.4,0.0,7.8,6.6,...,740.0,1.1,40.0,270.0,240.0,510.0,3.3,0.2,3.3,2.2
3,"Almond, dry roasted, salted",nuts,637.0,52.5,4.1,33.1,13.0,0.0,10.1,4.9,...,710.0,230.0,1190.0,270.0,280.0,470.0,3.7,2.4,3.3,2.0
4,"Almond, roasted, salted",nuts,649.0,55.2,4.2,34.8,13.5,0.0,7.2,4.6,...,670.0,330.0,1190.0,240.0,270.0,470.0,3.3,2.4,3.1,2.0


## Check data

In [5]:
# check columns in data
dataset.columns

Index(['name', 'category', 'energy_kcal', 'fat_g', 'fatty_acids_sat_g',
       'fatty_acids_monounsat_g', 'fatty_acids_polyunsat_g', 'cholesterol_mg',
       'carbohydrates_g', 'sugars_g', 'starch_g', 'fibres_g', 'protein_g',
       'salt_g', 'alcohol_g', 'water_g', 'vit_A_activity_re_µg',
       'vit_A_activity_rae_µg', 'retinol_µg', 'beta_carotene_activity_µg',
       'beta_carotene_µg', 'vit_B1_mg', 'vit_B2_mg', 'vit_B6_mg', 'vit_B12_µg',
       'niacin_mg', 'folate_µg', 'panthotenic_acid_mg', 'vit_c_mg', 'vit_d_µg',
       'vit_e_activity_mg', 'potassium_mg', 'sodium_mg', 'chloride_mg',
       'calcium_mg', 'magnesium_mg', 'phosphorus_mg', 'iron_mg', 'iodide_µg',
       'zinc_mg', 'selenium_µg'],
      dtype='object')

In [6]:
# check number of traces
tr_count = dataset.apply(lambda x: x.value_counts().get('tr.', 0))
assert sum(tr_count)==0

In [7]:
# check that there are no negative values
count = dataset.astype(str).applymap(lambda x: x.count('<')).sum().sum()
assert count==0

In [8]:
# check value range in numerical columns
dataset.describe()

,energy_kcal,fat_g,fatty_acids_sat_g,fatty_acids_monounsat_g,fatty_acids_polyunsat_g,cholesterol_mg,carbohydrates_g,sugars_g,starch_g,fibres_g,...,potassium_mg,sodium_mg,chloride_mg,calcium_mg,magnesium_mg,phosphorus_mg,iron_mg,iodide_µg,zinc_mg,selenium_µg
count,1092.000000,1092.000000,1087.000000,1085.000000,1085.000000,1080.000000,1092.000000,1087.000000,1038.000000,1091.000000,...,1087.000000,1092.000000,1068.000000,1087.000000,1085.000000,1086.000000,1087.000000,1071.000000,1086.000000,360.000000
mean,213.647436,11.307784,4.035053,4.143689,2.158712,37.357408,16.653206,6.698438,9.637481,2.394868,...,261.741674,470.952565,682.579685,74.623000,33.370324,146.316114,1.446551,16.703925,1.319707,6.590278
std,175.773371,17.531052,7.127153,7.534827,6.548878,67.158825,22.017310,14.380763,17.186977,4.712662,...,294.263743,2913.897325,4421.460235,155.684721,55.757110,160.340805,2.803399,189.859451,1.558232,14.986452
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76.750000,0.600000,0.100000,0.100000,0.200000,0.000000,0.600000,0.100000,0.000000,0.000000,...,130.000000,9.075000,36.000000,9.800000,11.000000,43.000000,0.300000,1.000000,0.300000,0.775000
50%,158.000000,4.600000,1.400000,1.500000,0.700000,10.000000,7.000000,1.600000,0.100000,1.200000,...,200.000000,64.000000,96.000000,23.000000,18.000000,110.000000,0.700000,2.800000,0.700000,2.600000
75%,322.250000,13.725000,5.000000,5.000000,1.500000,63.000000,22.525000,5.700000,12.175000,2.800000,...,320.000000,350.000000,512.500000,68.000000,29.000000,190.000000,1.550000,6.700000,1.975000,7.775000
max,900.000000,100.000000,85.900000,68.400000,68.000000,1255.000000,100.000000,100.000000,88.000000,74.100000,...,4380.000000,39900.000000,59900.000000,1340.000000,520.000000,1900.000000,38.000000,5100.000000,18.000000,230.000000


## Save data

If all tests passed, we save the dataset in a `.csv` file in the `data` folder. Do not save the first column as it is just the index.

In [9]:
# set the index
dataset.index.name = "ID"

# save
path2store = path2data.replace("_raw.xlsx", ".csv")
dataset.to_csv(path2store)
print(f"Stored cleaned data in: {path2store}")


Stored cleaned data in: ../data/swiss_food_composition.csv
